In [17]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import kagglehub
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import models
from PIL import Image

In [9]:
path = kagglehub.dataset_download("atulyakumar98/pothole-detection-dataset")

normal_dir = os.path.join(path, 'normal')
potholes_dir = os.path.join(path, 'potholes')
print("Normal images path:", normal_dir)
print("Potholes images path:", potholes_dir)

Normal images path: /Users/sai/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4/normal
Potholes images path: /Users/sai/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4/potholes


In [22]:
print("Dataset downloaded to:", path)

Dataset downloaded to: /Users/sai/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4


In [18]:
class PotholeDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_names = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(image_path).convert("RGB")
        
        label = 0 if 'normal' in self.image_names[idx] else 1
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [23]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize((256, 256))
])

normal_dataset = PotholeDataset(image_dir=normal_dir, transform=transform)
potholes_dataset = PotholeDataset(image_dir=potholes_dir, transform=transform)

combined_dataset = normal_dataset + potholes_dataset

train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(combined_dataset, batch_size=32, shuffle=False)

In [16]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 2) # Match the number of classes

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/sai/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:04<00:00, 24.1MB/s]


In [ ]:
for epoch in range(num_epochs):
    print(f"Starting epoch {epoch+1}/{num_epochs}")
    runningLoss = 0.0
    correctPredictions = 0.0
    totalPredictions = 0.0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        print(f"Processing batch {batch_idx+1}/{len(train_loader)}")
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        runningLoss += loss.item()
        _, predicted = torch.max(outputs, 1)
        totalPredictions += labels.size(0)
        correctPredictions += (predicted == labels).sum().item()

    epochLoss = runningLoss / len(train_loader)
    epochAccuracy = (correctPredictions / totalPredictions) * 100
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epochLoss:.4f}, Accuracy: {epochAccuracy:.2f}%")
